In [20]:
#this will clean all of the lovely data and merge it into a coherent pandas dataframe before I output it all to SQL

import csv
import pandas as pd
import re
import datetime
import ftfy
import pprint

from sqlalchemy import *
from sqlalchemy_utils import *
import psycopg2

In [50]:
###RUN THIS FIRST: postgres -D "C:\Users\Natalia\Desktop\Insight Data Science\postgres_data"

db = 'hostel'
username = 'Natalia'
engine = create_engine('postgres://%s@localhost/%s'%(username,db))
print(engine.url)

postgres://Natalia@localhost/hostel


In [21]:
hosteladdress1 = pd.read_csv('hosteladdress20180924-221852.csv')
hosteladdress2 = pd.read_csv('hosteladdress20180924-224509.csv')

In [44]:
addressname = 'hosteladdress.csv'

In [ ]:
with open(addressname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hostelid','address'])
    for hostel in hosteladdress1.itertuples():
        temp = hostel.address.split("&quot;")
        if len(temp) > 1:
            temp = temp[1]
        else:
            temp = temp[0]
        print(temp)
        writer.writerow([hostel.hostelid, temp])
    
    for hostel in hosteladdress2.itertuples():
        temp = hostel.address.split("&quot;")
        if len(temp) > 1:
            temp = temp[1]
        else:
            temp = temp[0]
        print(temp)
        writer.writerow([hostel.hostelid, temp])

del writer

In [46]:
hostellinks1 = pd.read_csv('hostelinks20180924-211004.csv')
hostellinks2 = pd.read_csv('hostelinks20180924-211021.csv')

In [47]:
hostellinks = hostellinks1.append(hostellinks2, ignore_index=True)
hostellinks.to_csv("hostellinks.csv")
hostelreviews = pd.read_csv('hostelreviews.csv')
hosteladdress = pd.read_csv('hosteladdress.csv')

In [51]:
hostelreviews.to_sql('hostelreviews', engine, if_exists='replace')
hostellinks.to_sql('hostellinks', engine, if_exists='replace')
hosteladdress.to_sql('hosteladdress', engine, if_exists='replace')

In [94]:
q = text("SELECT * from hosteladdress r JOIN hostellinks l on r.hostelid = l.hostelid")
hostelset =  pd.DataFrame(engine.execute(q).fetchall())

In [95]:
chostelset.head()

,0,1,2,3,4,5,6,7,8
0,554,48151,"357 College Street, Toronto, Ontario",0,Planet Traveler Hostel,48151,Toronto,https://www.hostelworld.com/hosteldetails.php/...,None
1,555,3856,"76 Church St., Toronto",1,HI Toronto,3856,Toronto,https://www.hostelworld.com/hosteldetails.php/...,None
2,556,42418,5 Selby St,2,All Days Hostel,42418,Toronto,https://www.hostelworld.com/hosteldetails.php/...,None
3,557,92762,280 Augusta Avenue,3,College Backpackers,92762,Toronto,https://www.hostelworld.com/hosteldetails.php/...,None
4,558,71032,7 Clarence Square,4,The Clarence Park,71032,Toronto,https://www.hostelworld.com/hosteldetails.php/...,None


In [97]:
col=[0,3,5,8]
hostelset.drop(hostelset.columns[col], axis=1, inplace=True)

In [101]:
hostelset.columns = ["hostelid", "address", "name", "city", "url"]
hostelset.head()

,hostelid,address,name,city,url
0,48151,"357 College Street, Toronto, Ontario",Planet Traveler Hostel,Toronto,https://www.hostelworld.com/hosteldetails.php/...
1,3856,"76 Church St., Toronto",HI Toronto,Toronto,https://www.hostelworld.com/hosteldetails.php/...
2,42418,5 Selby St,All Days Hostel,Toronto,https://www.hostelworld.com/hosteldetails.php/...
3,92762,280 Augusta Avenue,College Backpackers,Toronto,https://www.hostelworld.com/hosteldetails.php/...
4,71032,7 Clarence Square,The Clarence Park,Toronto,https://www.hostelworld.com/hosteldetails.php/...


In [103]:
hostelset.to_sql('hostellinks', engine, if_exists='replace')

In [104]:
hostelset.to_csv('hostellinks-all.csv')

In [105]:
hostelset.shape

(892, 5)

In [124]:
q = text("SELECT * from hostelreviews r JOIN hostellinks l on r.hostelid = l.hostelid")
reviewset =  pd.DataFrame(engine.execute(q).fetchall())

In [125]:
reviewset.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0,0,Planet Traveler Hostel,48151,Toronto,2018-09-13 00:00:00,review11816582,Good location in the middle of the city. Catch...,10,10,...,10,10,10,10,0,48151,"357 College Street, Toronto, Ontario",Planet Traveler Hostel,Toronto,https://www.hostelworld.com/hosteldetails.php/...
1,1,1,Planet Traveler Hostel,48151,Toronto,2018-09-12 00:00:00,review11811012,"Secure, good staff, good atmosphere, lovely vi...",6,8,...,8,8,10,10,0,48151,"357 College Street, Toronto, Ontario",Planet Traveler Hostel,Toronto,https://www.hostelworld.com/hosteldetails.php/...
2,2,2,Planet Traveler Hostel,48151,Toronto,2018-09-12 00:00:00,review11809075,"Clean, very friendly staff and free activities...",8,8,...,6,10,10,10,0,48151,"357 College Street, Toronto, Ontario",Planet Traveler Hostel,Toronto,https://www.hostelworld.com/hosteldetails.php/...
3,3,3,Planet Traveler Hostel,48151,Toronto,2018-09-09 00:00:00,review11801051,This is the first and only time I stay at a ho...,6,10,...,6,6,8,10,0,48151,"357 College Street, Toronto, Ontario",Planet Traveler Hostel,Toronto,https://www.hostelworld.com/hosteldetails.php/...
4,4,4,Planet Traveler Hostel,48151,Toronto,2018-09-04 00:00:00,review11800849,Planet Traveller provided everything a tourist...,8,10,...,8,10,10,8,0,48151,"357 College Street, Toronto, Ontario",Planet Traveler Hostel,Toronto,https://www.hostelworld.com/hosteldetails.php/...


In [126]:
col=[0,1,3,15, 16,18, 19]
reviewset.drop(reviewset.columns[col], axis=1, inplace=True)

In [127]:
reviewset.columns = ["name", "city", "date", "reviewid", "reviewtext",'value', 'security', 'location', 'facilities', 'staff',
       'atmosphere', 'cleanliness',"address","url"]

In [128]:
reviewset.head()

,name,city,date,reviewid,reviewtext,value,security,location,facilities,staff,atmosphere,cleanliness,address,url
0,Planet Traveler Hostel,Toronto,2018-09-13 00:00:00,review11816582,Good location in the middle of the city. Catch...,10,10,10,10,10,10,10,"357 College Street, Toronto, Ontario",https://www.hostelworld.com/hosteldetails.php/...
1,Planet Traveler Hostel,Toronto,2018-09-12 00:00:00,review11811012,"Secure, good staff, good atmosphere, lovely vi...",6,8,6,8,8,10,10,"357 College Street, Toronto, Ontario",https://www.hostelworld.com/hosteldetails.php/...
2,Planet Traveler Hostel,Toronto,2018-09-12 00:00:00,review11809075,"Clean, very friendly staff and free activities...",8,8,8,6,10,10,10,"357 College Street, Toronto, Ontario",https://www.hostelworld.com/hosteldetails.php/...
3,Planet Traveler Hostel,Toronto,2018-09-09 00:00:00,review11801051,This is the first and only time I stay at a ho...,6,10,8,6,6,8,10,"357 College Street, Toronto, Ontario",https://www.hostelworld.com/hosteldetails.php/...
4,Planet Traveler Hostel,Toronto,2018-09-04 00:00:00,review11800849,Planet Traveller provided everything a tourist...,8,10,10,8,10,10,8,"357 College Street, Toronto, Ontario",https://www.hostelworld.com/hosteldetails.php/...


In [129]:
reviewset.to_csv('hostelreviews-all.csv')

In [130]:
reviewset.to_sql('hostelreviews', engine, if_exists='replace')